In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from itertools import chain
import re
import json
from tqdm.autonotebook import tqdm
import string
import collections
from textblob import TextBlob
from pathlib import Path
import collections
import pickle

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import utils
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import plot_model

import en_core_web_lg
import spacy
from spacy.lang.en import English
from spacy.util import minibatch, compounding
from spacy import displacy

import warnings
warnings.filterwarnings('ignore')

# Functions to Prepare Data

In [ ]:
#define stopwords
from nltk.corpus import stopwords

stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']

In [ ]:
def clean_text(txt):
     return re.sub('[^A-Za-z0-9.]+', ' ', str(txt).lower()) 

In [ ]:
def text_cleaning(text, flg_stemm = False, flg_lemm = True, lst_stopwords = None):
    '''
    Converts all text to lower case, tokenize, remove multiple spaces, stopwords, stemming, lemmatize, 
    then convert all back to string
    
    text: string - name of column containing text
    lst_stopwords: list - list of stopwords to remove
    flg_stemm: bool - whether stemming is to be applied
    flg_lemm: bool - whether lemmitisation is to be applied
    '''
    
    #clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    #tokenize (convert from string to list)
    lst_text = text.split()
    
    #remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    stopwords_list]
                
    #stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    #lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    #back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
MAX_LENGTH = 64
OVERLAP = 20
    
def shorten_sentences(sentences):
    """
    Sentences that have more than MAX_LENGTH words will be split
    into multiple sentences with overlappings.
    """
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

# OBTAIN

In [ ]:
#define paths
os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data/')
train_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
#create a function to get the text from the JSON file and append it to the new column in table
def read_json_pub(filename, train_path = train_path, output = 'text'):
    json_path = os.path.join(train_path, (filename + '.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
#read 
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

#review
train.head()

# PREPROCESSING

In [ ]:
#https://www.kaggle.com/jagdmir/spacy-ner-model
import nltk

DATA = []
ent_count = 0
empty_count = 0

for idx,row in tqdm(train.iterrows()):
    pub = '../input/coleridgeinitiative-show-us-the-data/train/' + row.Id + '.json'            
    f = open(pub)  
    data = json.load(f)      
    
    balanced = False

    sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(data))]
    sentences = shorten_sentences(sentences) # make sentences short
    sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
    for sentence in sentences:          

        a = re.search(row.cleaned_label.lower(), sentence)
        b = re.search(row.dataset_label.lower(), sentence)
        c = re.search(row.dataset_title.lower(), sentence)
        
        if  a != None:
            DATA.append((sentence, {"entities":[(a.span()[0], a.span()[1], "DATASET")]}))
            ent_count = ent_count + 1
            balanced = True
            
        elif b != None:
            DATA.append((sentence, {"entities":[(b.span()[0], b.span()[1], "DATASET")]}))
            ent_count = ent_count + 1
            balanced = True
            
        elif c != None:
            DATA.append((sentence, {"entities":[(c.span()[0], c.span()[1], "DATASET")]}))
            ent_count = ent_count + 1
            balanced = True
            
        else:
            if balanced:
                DATA.append((sentence, {'entities':[]}))
                empty_count = empty_count + 1
                balanced = False
                
print('Text with entities:', ent_count)
print('Text without entities:', empty_count)

In [ ]:
#print an example of a sentence
print(DATA[0][0])

print('\n')
#print an example of the sentence's entities
print(DATA[0][1])

In [ ]:
#print an example of a sentence
print(DATA[10][0])

print('\n')
#print an example of the sentence's entities
print(DATA[10][1])

In [ ]:
train_df = pd.DataFrame(DATA)
train_df = train_df.rename({0: 'Sentence', 1: 'Entities'}, axis = 1)

#review
train_df.tail(10)

# spaCy Train with 'DATASET' Entity

In [ ]:
import random
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path

def train_spacy(DATA, iterations, model):
    
    print(f"downloads = {model}")
    if model is not None:
        print(f"training existing model")
        nlp = spacy.load(model)
        print("Model is Loaded '%s'" % model)
    else:
        print(f"Creating new model")

        nlp = spacy.blank('en')  #create blank language class

    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last = True)
    else:
        ner = nlp.get_pipe('ner')

    #based on template, get labels and save those for further training   
    for _, annotations in DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])           #add 'DATASET' entities

    #get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER, prevent the other components from being involved
        if model is None:
            optimizer = nlp.begin_training()
        else:
            optimizer = nlp.entity.create_optimizer()
        tags = dict()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(DATA)             #shuffle the examples randomly
            losses = {}
            batches = minibatch(DATA, size = compounding(4.0, 16.0, 1.001))
            
            #type 2 with mini batch
            for batch in batches:                
                texts, annotations = zip(*batch)                
                golds = annotations          #zip method
                nlp.update(texts,            #batch of texts
                           golds,            #batch of annotations
                           drop = 0.2,       #dropout - make it harder to memorise data
                           losses = losses,  #hold the losses against each pipeline component
                           sgd = optimizer)
            print(losses)
            
    return nlp

In [ ]:
start = datetime.datetime.now()

#train
nlp = train_spacy(DATA, 20, "en_core_web_sm") 

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
text = 'This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment.'

#display sentence involving original entities
spacy.displacy.render(nlp(text), style = "ent")

In [ ]:
text = 'the international community came together to build the rsna international covid open radiology database ricord 1 which will be accessible to all investigator to further scientific knowledge and facilitate the development of rapid quantitative assessment tool'
#display sentence involving original entities
spacy.displacy.render(nlp(text), style = "ent")

In [ ]:
text = 'the data utilized were derived from the data base our world in data 32 and had the form of cumulative infection and death data for country around the worldnin figure 2 the cumulative graph of the registered covid19infected population in france appears'
#display sentence involving original entities
spacy.displacy.render(nlp(text), style = "ent")

In [ ]:
text = 'the data utilized were derived from the data base our world in data 32 and had the form of cumulative infection and death data for country around the worldnin figure 2 the cumulative graph of the registered covid19infected population in france appears'

doc = nlp(text)

for entity in doc.ents:
    print(entity.label_, ' | ', entity.text)

### Visualize

In [ ]:
text = 'This study used data from the National Education Longitudinal Study (NELS:88) to examine the effects of dual enrollment programs for high school students on college degree attainment.'

displacy.render(nlp(str(text)), style='dep', jupyter = True, options = {'distance': 120})

# EVALUATION

In [ ]:
#dictionary which will be populated with the entities and result information
entity_evaluation = {}

# helper function to udpate the entity_evaluation dictionary
def update_results(entity, metric):
    if entity not in entity_evaluation:
        entity_evaluation[entity] = {"correct": 0, "total": 0}
    
    entity_evaluation[entity][metric] += 1

#same as before, see if entities from test set match what spaCy currently predicts
for data in DATA:
    sentence = data[0]
    entities = data[1]["entities"]

    for entity in entities:
        doc = nlp(sentence)
        correct_text = sentence[entity[0]:entity[1]]

        for ent in doc.ents:
            if ent.label_ == entity[2] and ent.text == correct_text:
                update_results(ent.label_, "correct")
                break

        update_results(entity[2], "total")

In [ ]:
sum_total = 0
sum_correct = 0

for entity in entity_evaluation:
    total = entity_evaluation[entity]["total"]
    correct = entity_evaluation[entity]["correct"]

    sum_total += total
    sum_correct += correct
    
    print("{} | {:.2f}%".format(entity, correct / total * 100))

print()
print("Overall accuracy: {:.2f}%".format(sum_correct / sum_total * 100))

In [ ]:
#save
nlp.meta["name"] = "dataset_entity_extractor"
nlp.to_disk("./dataset_entity_extractor")

In [ ]:
import pickle

pickle.dump(nlp, open( "dataset_entity_extractor.p", "wb" ))

# PREDICTION

In [ ]:
#get text
tqdm.pandas()
train['text'] = train['Id'].progress_apply(read_json_pub)

#clean text
train['text'] = train['text'].progress_apply(clean_text)

In [ ]:
from functools import partial

#read data
sample_submission = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')

#apply the function to submission data
tqdm.pandas()
sample_submission['text'] = sample_submission['Id'].progress_apply(partial(read_json_pub, train_path = test_path))

#review
sample_submission.head()

In [ ]:
temp_1 = [x.lower() for x in train['dataset_label'].unique()]
temp_2 = [x.lower() for x in train['dataset_title'].unique()]
temp_3 = [x.lower() for x in train['cleaned_label'].unique()]

existing_labels = set(temp_1 + temp_2 + temp_3)

In [ ]:
literal_matching = True
spacy_prediction = True

id_list = []
labels_list = []

for index, row in tqdm(sample_submission.iterrows()):

    sample_text = row['text']

    row_id = row['Id']
    
    #takes only the rows where train file is identical to a test file
    temp_df = train[train['text'] == clean_text(sample_text)]
    cleaned_labels = temp_df['cleaned_label'].to_list()
    
    #literal_matching 
    if literal_matching:
        for known_label in existing_labels:
            if known_label in sample_text.lower():    
                cleaned_labels.append(clean_text(known_label))

        print('cleaned label:', set(cleaned_labels))
        
    #spaCy NER 
    if spacy_prediction:
        
        #extract sentences
        sentences = [clean_text(sentence) for sentence in nltk.sent_tokenize(str(sample_text))]
        sentences = shorten_sentences(sentences) # make sentences short
        sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars
    
        #for each entity, use our updated model to make a prediction on the sentence
        for sentence in sentences:
            
            doc = nlp(sentence)

            for ent in doc.ents:
                spacy_labels = ent.text
                
        print('spacy label:', spacy_labels)

    cleaned_labels.append(spacy_labels)
        
    cleaned_labels = set(cleaned_labels)
    cleaned_labels = [clean_text(x) for x in cleaned_labels]    
    labels_list.append('|'.join(cleaned_labels))
    print('label list:', labels_list)   
    id_list.append(row_id)
    print('\n')

In [ ]:
#get dataframe
sample_submission['PredictionString'] = labels_list
sample_submission

In [ ]:
print(sample_submission['PredictionString'][0])
print('\n')
print(sample_submission['PredictionString'][1])
print('\n')
print(sample_submission['PredictionString'][2])
print('\n')
print(sample_submission['PredictionString'][3])

In [ ]:
#drop 'text'
sample_submission.drop(columns = 'text', axis = 1, inplace = True)
sample_submission

In [ ]:
#save
sample_submission.to_csv('submission.csv', index = False)

#check
submission = pd.read_csv('submission.csv')
submission